In [1]:
import pandas as pd
import numpy as np

In [2]:
import tqdm

In [3]:
baseData = pd.read_csv("../dataset/train_ver2.csv")

C:\Users\caose\AppData\Local\Temp\ipykernel_69360\2862590443.py:1: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  baseData = pd.read_csv("../dataset/train_ver2.csv")


In [4]:
### 3개월 데이터로 줄인다. 값이 너무 많아서 최근의 추세로 판단하겠다

In [5]:
base1 = baseData.loc[baseData.fecha_dato == "2016-04-28"]

In [6]:
base2 = baseData.loc[baseData.fecha_dato == "2016-05-28"]

In [7]:
data1 = base1.append(base2, ignore_index=True)

C:\Users\caose\AppData\Local\Temp\ipykernel_69360\4183799191.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data1 = base1.append(base2, ignore_index=True)


In [8]:
data1 = data1.reset_index(drop=True)

In [9]:
# 기존
data1 = data1.drop(['fecha_dato', 'ncodpers','ind_empleado','pais_residencia','fecha_alta','ind_nuevo','indrel','ult_fec_cli_1t',\
                    'conyuemp','canal_entrada','indfall','tipodom','cod_prov','nomprov','ind_actividad_cliente',\
                    'ind_aval_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1',\
                    'ind_ctpp_fin_ult1','ind_ecue_fin_ult1','ind_hip_fin_ult1','indresi',\
                    'ind_reca_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1',\
                    'ind_cco_fin_ult1','ind_cder_fin_ult1',"antiguedad",'indrel_1mes','tiprel_1mes',\
                    'ind_deco_fin_ult1','ind_deme_fin_ult1','ind_ahor_fin_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1'], axis=1)

In [10]:
oneStepDf = data1.copy()

# 칼럼 처리

### 나이

In [11]:
oneStepDf["age"] = oneStepDf["age"].astype(int)

In [12]:
oneStepDf["age"] = np.where(oneStepDf.age <= 9,0,
                   np.where(oneStepDf.age <= 19,1,
                   np.where(oneStepDf.age <= 29,2,
                   np.where(oneStepDf.age <= 39,3,
                   np.where(oneStepDf.age <= 49,4,
                   np.where(oneStepDf.age <= 59,5,
                   np.where(oneStepDf.age <= 69,6,
                   np.where(oneStepDf.age <= 79,7,
                   np.where(oneStepDf.age <= 89,8,
                   np.where(oneStepDf.age <= 99,9,10))))))))))

### 총소득

In [13]:
oneStepDf["renta"].fillna(oneStepDf.groupby('age')["renta"].transform('median'), inplace=True)

In [14]:
oneStepDf["renta"] = oneStepDf["renta"].astype(int)

In [15]:
oneStepDf["renta"].describe().astype(int)

count     1859727
mean       125652
std        202483
min          1202
25%         78250
50%         96544
75%        133053
max      28894395
Name: renta, dtype: int32

In [16]:
oneStepDf["renta"] = np.where(oneStepDf.renta <= oneStepDf['renta'].quantile(0.2), 1,
                     np.where(oneStepDf.renta <= oneStepDf['renta'].quantile(0.4), 2,
                     np.where(oneStepDf.renta <= oneStepDf['renta'].quantile(0.6), 3,
                     np.where(oneStepDf.renta <= oneStepDf['renta'].quantile(0.8), 4, 5))))

In [17]:
oneStepDf["renta"].drop_duplicates()

0     5
1     4
2     1
4     3
28    2
Name: renta, dtype: int32

### 성별

In [18]:
oneStepDf.sexo.drop_duplicates()

0           H
2           V
302702    NaN
Name: sexo, dtype: object

In [ ]:
len(oneStepDf.loc[oneStepDf.sexo == "H"])

In [ ]:
len(oneStepDf.loc[oneStepDf.sexo == "V"])

In [19]:
oneStepDf.sexo = oneStepDf.sexo.fillna("H")

In [20]:
oneStepDf.sexo.drop_duplicates()

0    H
2    V
Name: sexo, dtype: object

In [21]:
dict = {"V":0, "H":1}

oneStepDf.sexo = oneStepDf.sexo.str.upper()
oneStepDf.sexo = oneStepDf.sexo.map(dict)

In [22]:
oneStepDf.sexo = oneStepDf.sexo.astype(int)

In [23]:
oneStepDf.sexo.dtype

dtype('int32')

### 외국인 지수

In [24]:
dict = {"N":0, "S":1}

oneStepDf.indext = oneStepDf.indext.str.upper()
oneStepDf.indext = oneStepDf.indext.map(dict)

In [25]:
oneStepDf.indext.unique()

array([0, 1], dtype=int64)

### 고객 분류

In [26]:
oneStepDf.segmento.drop_duplicates()

0       02 - PARTICULARES
4      03 - UNIVERSITARIO
5                01 - TOP
922                   NaN
Name: segmento, dtype: object

In [27]:
oneStepDf.segmento = np.where(oneStepDf.segmento == '03 - UNIVERSITARIO', 0, 
                     np.where(oneStepDf.segmento == '01 - TOP', 1,
                     np.where(oneStepDf.segmento == '02 - PARTICULARES', 1, oneStepDf.segmento)))

In [28]:
oneStepDf.loc[oneStepDf["age"] < 2, 'segmento'] = oneStepDf.segmento.fillna(0)

In [29]:
oneStepDf.loc[oneStepDf["age"] >= 2, 'segmento'] = oneStepDf.segmento.fillna(1)

In [30]:
oneStepDf.segmento = oneStepDf.segmento.astype(int)

In [31]:
oneStepDf.segmento.dtype

dtype('int32')

In [32]:
oneStepDf.dtypes

sexo                 int32
age                  int32
indext               int64
renta                int32
segmento             int32
ind_cno_fin_ult1     int64
ind_ctju_fin_ult1    int64
ind_dela_fin_ult1    int64
ind_fond_fin_ult1    int64
ind_tjcr_fin_ult1    int64
ind_valo_fin_ult1    int64
dtype: object

### 나이

In [33]:
oneStepDf["age"] = np.where(oneStepDf.age <= 1,1, # 미성년
                   np.where(oneStepDf.age <= 4,2, # 청년
                   np.where(oneStepDf.age <= 6,3,4))) # 중년, 노년

In [34]:
oneStepDf["age"].drop_duplicates()

0    4
1    3
2    2
8    1
Name: age, dtype: int32

In [35]:
oneStepDf = oneStepDf.reset_index(drop=True)

In [36]:
# label이 될 컬럼을 상품명으로 변경

oneStepDf.loc[oneStepDf['ind_dela_fin_ult1'] == 1, 'ind_dela_fin_ult1'] = '예금'
oneStepDf.loc[oneStepDf['ind_cno_fin_ult1'] == 1, 'ind_cno_fin_ult1'] = '급여계좌'
oneStepDf.loc[oneStepDf['ind_ctju_fin_ult1'] == 1, 'ind_ctju_fin_ult1'] = '주니어계좌'
oneStepDf.loc[oneStepDf['ind_fond_fin_ult1'] == 1, 'ind_fond_fin_ult1'] = '펀드계좌'
oneStepDf.loc[oneStepDf['ind_tjcr_fin_ult1'] == 1, 'ind_tjcr_fin_ult1'] = '신용카드'
oneStepDf.loc[oneStepDf['ind_valo_fin_ult1'] == 1, 'ind_valo_fin_ult1'] = '증권'

In [37]:
oneStepDf

,sexo,age,indext,renta,segmento,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_dela_fin_ult1,ind_fond_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1
0,1,4,0,5,1,0,0,0,0,0,0
1,1,3,0,4,1,0,0,0,0,0,0
2,0,2,0,1,1,0,0,0,0,0,0
3,1,4,0,5,1,0,0,0,0,0,0
4,0,2,0,3,0,급여계좌,0,0,0,신용카드,0
...,...,...,...,...,...,...,...,...,...,...,...
1859722,0,2,0,1,0,0,0,0,0,0,0
1859723,0,2,0,1,0,0,0,0,0,0,0
1859724,1,2,0,3,1,0,0,0,0,0,0
1859725,1,2,0,5,0,0,0,0,0,0,0


In [38]:
oneStepDf.columns

Index(['sexo', 'age', 'indext', 'renta', 'segmento', 'ind_cno_fin_ult1',
       'ind_ctju_fin_ult1', 'ind_dela_fin_ult1', 'ind_fond_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1'],
      dtype='object')

In [39]:
# 라벨부분 컬럼
expandColumn = ['ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_dela_fin_ult1',\
                'ind_fond_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1']

In [40]:
# 컬럼
dataCol = ['sexo', 'age', 'indext', 'renta', 'segmento']

In [41]:
oneStepDf

,sexo,age,indext,renta,segmento,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_dela_fin_ult1,ind_fond_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1
0,1,4,0,5,1,0,0,0,0,0,0
1,1,3,0,4,1,0,0,0,0,0,0
2,0,2,0,1,1,0,0,0,0,0,0
3,1,4,0,5,1,0,0,0,0,0,0
4,0,2,0,3,0,급여계좌,0,0,0,신용카드,0
...,...,...,...,...,...,...,...,...,...,...,...
1859722,0,2,0,1,0,0,0,0,0,0,0
1859723,0,2,0,1,0,0,0,0,0,0,0
1859724,1,2,0,3,1,0,0,0,0,0,0
1859725,1,2,0,5,0,0,0,0,0,0,0


### 11개 상품 1개의 컬럼으로 수정

In [42]:
wholeList = []
for i in tqdm.tqdm(range(len(oneStepDf))):
    for j in expandColumn:
        rowList = []
        rowList.append(oneStepDf[j][i])
        for k in dataCol:
            rowList.append(oneStepDf[k][i])
        wholeList.append(rowList)
secondStepDf = pd.DataFrame(wholeList)

100%|██████████████████████████████████████████████████████████████████████| 1859727/1859727 [06:43<00:00, 4605.39it/s]


In [43]:
secondStepDf.rename(columns={0:"FINANCIALPRODUCT"}, inplace=True)
secondStepDf.rename(columns={1:"GENDER"}, inplace=True)
secondStepDf.rename(columns={2:"AGE"}, inplace=True)
secondStepDf.rename(columns={3:"CUST_TYPE"}, inplace=True)
secondStepDf.rename(columns={4:"INCOME"}, inplace=True)
secondStepDf.rename(columns={5:"FOREINERINDEX"}, inplace=True)

,FINANCIALPRODUCT,GENDER,AGE,CUST_TYPE,INCOME,FOREINERINDEX
0,0,1,4,0,5,1
1,0,1,4,0,5,1
2,0,1,4,0,5,1
3,0,1,4,0,5,1
4,0,1,4,0,5,1
...,...,...,...,...,...,...
11158357,0,1,2,0,3,1
11158358,0,1,2,0,3,1
11158359,0,1,2,0,3,1
11158360,0,1,2,0,3,1


In [44]:
# 가입 안 된 데이터 삭제
secondStepDf = secondStepDf.loc[secondStepDf.FINANCIALPRODUCT != 0]
secondStepDf = secondStepDf.reset_index(drop=True)

,FINANCIALPRODUCT,GENDER,AGE,CUST_TYPE,INCOME,FOREINERINDEX
0,급여계좌,0,2,0,3,0
1,신용카드,0,2,0,3,0
2,신용카드,0,2,0,5,1
3,증권,0,2,0,5,1
4,급여계좌,1,2,0,1,1
...,...,...,...,...,...,...
365515,급여계좌,0,2,0,5,0
365516,주니어계좌,1,1,0,5,1
365517,주니어계좌,0,1,0,5,1
365518,급여계좌,1,2,0,2,0


In [45]:
from sklearn.preprocessing import LabelEncoder

In [46]:
labelEn = LabelEncoder()

In [47]:
secondStepDf.FINANCIALPRODUCT = labelEn.fit_transform(secondStepDf.FINANCIALPRODUCT)

In [48]:
secondStepDf.corr()

,FINANCIALPRODUCT,GENDER,AGE,CUST_TYPE,INCOME,FOREINERINDEX
FINANCIALPRODUCT,1.000000,-0.071372,0.129956,-0.062945,0.124151,0.192785
GENDER,-0.071372,1.000000,-0.058006,0.027726,0.005822,-0.061043
AGE,0.129956,-0.058006,1.000000,-0.039967,0.113930,0.174993
CUST_TYPE,-0.062945,0.027726,-0.039967,1.000000,-0.025873,-0.011901
INCOME,0.124151,0.005822,0.113930,-0.025873,1.000000,0.070962
FOREINERINDEX,0.192785,-0.061043,0.174993,-0.011901,0.070962,1.000000


In [49]:
secondStepDf.FINANCIALPRODUCT = labelEn.inverse_transform(secondStepDf.FINANCIALPRODUCT)

In [50]:
secondStepDf

,FINANCIALPRODUCT,GENDER,AGE,CUST_TYPE,INCOME,FOREINERINDEX
0,급여계좌,0,2,0,3,0
1,신용카드,0,2,0,3,0
2,신용카드,0,2,0,5,1
3,증권,0,2,0,5,1
4,급여계좌,1,2,0,1,1
...,...,...,...,...,...,...
365515,급여계좌,0,2,0,5,0
365516,주니어계좌,1,1,0,5,1
365517,주니어계좌,0,1,0,5,1
365518,급여계좌,1,2,0,2,0


In [52]:
### csv저장

In [53]:
secondStepDf.to_csv("../dataset/Financial_Product_Recommendation.csv", index=False, mode ="w")